# 4. Etude du jeu de données Diabetes

Nous allons étudier un problème de regression sur un jeu de données de patients atteints de diabète. Nous utiliserons un jeu de données qui décrit 442 patients souffrant de diabète. Il s'agit de prédire l'évolution de la maladie au bout d'un an. Commencons par charger ce dataset et afficher sa description :

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np

diabetes = datasets.load_diabetes()
df = pd.DataFrame(diabetes.data)
df.columns = diabetes.feature_names
df["target"] = diabetes.target

In [ ]:
print(diabetes.DESCR)

In [ ]:
df

## 6.1 Analyse exploratoire des données

Réalisez une analyse exploratoire de ce dataset, pour vous aider vous pouvez répondre aux questions suivantes :
* Quels sont les types de variables du dataset ?
* Quelles sont les modalités des variables qualitatives ? (aka : catégorielles)
* Quels sont les moyennes, écarts types, min, max ... des variables numériques ?
* Existe-t'il des corrélations fortes entre ces variables numériques ?

Vous pouvez aussi afficher certains graphiques avec matplotlib, seaborn ou plotly. Par exemple : [`sns.distplot`](https://seaborn.pydata.org/generated/seaborn.distplot.html), [`sns.boxplot`](https://seaborn.pydata.org/generated/seaborn.boxplot.html) ou encore [`sns.heatmap`](https://seaborn.pydata.org/generated/seaborn.heatmap.html).

Pensez aussi à regarder certains individus de ce dataset (pas seulement des informations statistiques).

In [ ]:
import math, pprint
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
from matplotlib.colors import ListedColormap

pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

def print_df(df):
    display(HTML(df.to_html()))
    
def boxplot_grid(df, nb_grid_cols=3, figsize=(16,15)):
    df_num = df.select_dtypes(include=[np.number])
    nb_num_vars = len(df_num.columns)
    fig, axes = plt.subplots(math.ceil(nb_num_vars/nb_grid_cols), nb_grid_cols, figsize=figsize)
    for i, col in enumerate(df_num.columns):
        sns.boxplot(df_num[col], orient="v", ax=axes[int(i/nb_grid_cols),i%nb_grid_cols])
    plt.show()
        
def density_plot_grid(df, nb_grid_cols=3, figsize=(16,15)):
    df_num = df.select_dtypes(include=[np.number])
    nb_num_vars = len(df_num.columns)
    fig, axes = plt.subplots(math.ceil(nb_num_vars/nb_grid_cols), nb_grid_cols, figsize=figsize)
    for i, col in enumerate(df_num.columns):
        not_null_col = df_num[col][df_num[col].notnull()]
        sns.distplot(not_null_col, ax=axes[int(i/nb_grid_cols),i%nb_grid_cols])
    plt.show()

def explore_data(df):
    print(f"Dimensions de la matrice des observations : {df.shape}")
    print("\nExtrait : \n")
    print_df(df.head())
    print("\nTypes des variables : \n")
    print(df.dtypes)
    print("\nDescription des variables numériques : \n")
    print_df(df.describe())
    print("\nDescription des variables catégorielles : \n")
    for col in df.select_dtypes(include=[object]).columns:
        counts = df[col].value_counts()
        if len(counts) < 20:
            print("\nModalités de la variable ", col)
            pprint.pprint(df[col].value_counts())
        else:
            print("\nLa variable %s possède %i modalités" % (col, len(counts)))
    print("\nAnalyse des corrélations : \n")
    plt.figure(figsize=(6, 5))
    sns.heatmap(df.corr())
    plt.show()
    print("\nValeurs manquantes : \n")
    print(pd.isnull(df).sum())
    print("\nBoîtes à moustaches : \n")
    boxplot_grid(df)
    print("\nHistogrammes avec estimation de densité : \n")
    density_plot_grid(df)

explore_data(df)

## 6.2 Construction du jeu d'entraînement et du jeu de test

Nous devons sans tarder réserver un jeu de test... Sinon, nous serons incapables de mesurer la capacité de votre modèle à généraliser, c'est-à-dire à prédire sur de nouvelles données. Pour cela utilisez la fonction [`sklearn.model_selection.train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) en réservant 20% du dataset pour notre jeu de test :

In [ ]:
X = df[df.columns.difference(['target'])]
y = df['target']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

In [ ]:
y_test

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=77)

In [ ]:
test

Avant de construire les modèles prédictifs, il convient de séparer les variables de la cible à prédire. construisez deux dataframes `X_tain` et `X_test` contenant notre jeu de donnée sans la cible (colonne `target`). De même construisez `y_train` et `y_test` ne contenant que la cible :

In [ ]:
X_train = train.drop("target", axis=1)
y_train = train["target"].copy()
X_test = test.drop("target", axis=1)
y_test = test["target"].copy()

## 6.3 Régression linéaire

Nous allons maintenant entrainer un modèle de régression linéaire sur le jeu d'entrainement et vérifier ses performances sur le jeu de test.

Importez la classe `LinearRegression` du module `sklearn.linear_model` ([sklearn.linear_model.LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)) et la fonction `mean_squared_error` du module `sklearn.metrics` ([sklearn.metrics.mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)) :

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Créez une instance de la classe `LinearRegression` et affectez là à la variable `lr` :

In [ ]:
lr = LinearRegression()

Entrainez votre modèle de régression linéaire sur votre dataset d'entrainement. En clair, appelez la méthode `fit` sur votre objet `lr` en lui passant comme arguments votre dataset d'entrainement (`X_train`, car nous ne voulons pas lui donner la cible) et la cible (i.e. : les labels, `y_train`). L'appel à `fit` ne retourne rien, votre objet `lr` a évolué en interne et à trouvé les coefficients de la régression linéaire minimisant la fonction de coût :

In [ ]:
lr.fit(X_train, y_train)

Affichez le contenu des attributs `intercept_` et `coef_` de votre objet `lr`, ce sont, respectivement, l'ordonnée à le biais de votre régresseur (nous allons expliquer ce biais ci-dessous).

In [ ]:
lr.intercept_, lr.coef_

Importez le package `numpy` dans son intégralité et renomez le en `np` (syntaxe `import le_nom_du_package as son_alias`) :

In [ ]:
import numpy as np

Calculez la moyenne des cibles en appelant la fonction `mean` de `numpy` et en lui passant `y_train` en argument. Cette valeur devrait être égale au biais de notre régression linéaire. Ce biais est la valeur que doit prendre la cible si tous les coefficients étaient à zéro (un tel individu n'est probablement pas possible dans la réalité), ici, la valeur de `intercept_` est la moyenne des valeurs de nos cibles :

In [ ]:
np.mean(y_train)

## 6.4 Mesures de performance

Vérifions les performances de notre regresseur sur notre dataset de test. Appelez la méthode `predict` de votre objet `lr` avec comme argument `X_test` seulement (nous ne lui fournissons pas la cible de notre dataset de test, car c'est que nous voulons prédire). Stockez ce résultat dans une variable `y_pred` et affichez le contenu de cette variable. Que vous renvoie cette méthode ?

In [ ]:
y_pred = lr.predict(X_test)
y_pred

Importez les méthodes `mean_squared_error` et `mean_absolute_error` du module [`sklearn.metrics`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics), vous pouvez importer plusieurs fonctions (ou classes) d'un même module en utilisant la syntaxe `from module_name import function_name_a, function_name_b` :

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

Calculez et affichez l'erreur quadratique moyenne (Mean Squarred Error ou MSE) de vos prédictions sur le dataset de test en utilisant la fonction `mean_squared_error` avec `y_test` et `y_pred` (dans cet ordre). Est-ce que cette valeur vous semble interprétable par rapport à la moyenne (fonction `mean` du module `numpy`), le max (fonction `max` du module `numpy` ou fonction `max` de la librairie standard de Python) et le min (fonction `min` du module `numpy` ou fonction `min` de la librairie standard Pyton) de votre cible ?

In [ ]:
np.min(y_test), np.max(y_test), np.mean(y_test)

In [ ]:
mean_squared_error(y_test, y_pred)

Importez l'ensemble du module `math` de la librairie standard de Python (syntaxe : `import module_name`) :

In [ ]:
import math

Appliquez la fonction `sqrt` du module `math` la librairie standard de Python (syntaxe : `nom_module.nom_de_la_fonction`) sur le MSE (vous obtiendez alors le Root MSE ou RMSE), est-ce que cette valeur vous semble interprétable par rapport à votre cible ?

In [ ]:
math.sqrt(mean_squared_error(y_test, y_pred))

Calculez et affichez l'erreur absolue moyenne (Mean Absolute Error ou MAE) de vos prédictions sur le dataset de test en utilisant la fonction `mean_absolute_error` avec `y_test` et `y_pred` (dans cet ordre), est-ce que cette valeur vous semble interprétable par rapport à votre cible ?

In [ ]:
mean_absolute_error(y_test, y_pred)

Pour mieux comprendre la différence entre la RMSE et la MAE, appliquez ces deux métriques sur les trois ensembles de cibles de test et de prédiction synthétiques suivants. Quelle conclusion pouvez-vous en tirer sur la différence entre RMSE et MAE ?

In [ ]:
y_synthetic_test = [1, 1, 1, 1, 1, 1, 1]
y_synthetic_pred_1 = [3, 3, 3, 3, 3, 3, 3]
y_synthetic_pred_2 = [2, 2, 2, 2, 3, 4, 6]
y_synthetic_pred_3 = [1, 1, 1, 1, 1, 1, 15]

In [ ]:
mean_absolute_error(y_synthetic_test, y_synthetic_pred_1)

In [ ]:
math.sqrt(mean_squared_error(y_synthetic_test, y_synthetic_pred_1))

In [ ]:
mean_absolute_error(y_synthetic_test, y_synthetic_pred_2)

In [ ]:
math.sqrt(mean_squared_error(y_synthetic_test, y_synthetic_pred_2))

In [ ]:
mean_absolute_error(y_synthetic_test, y_synthetic_pred_3)

In [ ]:
math.sqrt(mean_squared_error(y_synthetic_test, y_synthetic_pred_3))

## 6.5 Regression linéaire régularisée

Entrainez sur le dataset d'entrainement et testez les performances (RMSE et MAE) sur le dataset de test des modèles de régressions linéaire régularisés [`sklearn.linear_model.Ridge`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) et [`sklearn.linear_model.Lasso`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) en utilisant les paramètres par défaut (vous n'avez donc pas besoin de spécifier de paramètres lors de l'appel aux constructeurs de ces classes, par exemple : `ridge = Ridge()`). Entre la régression linéaire simple, la régression Ridge et la régression Lasso, quelle est la méthode la plus performante sur le dataset de test ?

In [ ]:
from sklearn.linear_model import Ridge, Lasso

In [ ]:
ridge = Ridge()
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)
mean_absolute_error(y_test, y_pred), math.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
lasso = Lasso()
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
mean_absolute_error(y_test, y_pred), math.sqrt(mean_squared_error(y_test, y_pred))

## 6.6 Validation croisée

Nous avons comparé l'efficacité de plusieurs modèles sur notre dataset de test ... ce qu'il ne faut surtout par faire. Nous reprenons cette comparaison en utilisant la validation croisée sur le jeu d'entrainement.

Créez un nouveau modèle de régression linéaire simple, puis appliquez la fonction [sklearn.model_selection.cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html). Les paramètres à utiliser pour cette fonction sont dans l'ordre : votre modèle de régression linéaire, votre dataset d'entrainement, vos cibles d'entrainement, `scoring="neg_mean_squared_error"` (nous devons indiquer la métrique de performance à calculer) et `cv=10` (vous pouvez changer ce paramètre pour obtenir plus ou moins de plis). Affectez le résultat à une variable `linear_cv_scores` et affichez la.

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
linear_cv_scores = cross_val_score(LinearRegression(), X_train, y_train, scoring="neg_mean_squared_error", cv=10)
linear_cv_scores

Nous obtenons la performance d'une régression linéaire pour 10 plis sur notre dataset d'entrainement. Cette valeur est négative car scikit-learn ne permet de calculer que la valeur négative du MSE avec une validation croisée.

Affichez le résultat de `-scores`. Qu'observez vous ?
Pour calculer le RMSE pour chacun des 10 plis, ous ne pouvons pas appliquer la méthode `sqrt` de la librairie standard de Python directement car elle n'est pas applicable à un tableau numpy (`type(scores)` vous permet de vérifier le type de cette variable). Appliquez la fonction `numpy.sqrt` à aux valeurs négatives de `scores` ou appelez directement la méthode `sqrt` sur la variable `scores` (tout tableau numpy dispose d'un ensemble de méthodes déjà disponibles, tel que la racine carrée).

Calculez ensuite la moyenne et l'écart type des RMSE obtenus sur les 10 plis (vous pouvez appeler directement les méthodes `mean` et `std` sur des tableaux numpy). Que pouvez-vous en conclure sur la  variance d'un modèle de régression linéaire appliqué à notre dataset ?

In [ ]:
-linear_cv_scores

In [ ]:
type(linear_cv_scores)

Calculez la moyenne (`numpy.mean`) et l'écart type (`numpy.std`) de `scores`. Que pouvez-vous conclure sur le biais et la variance d'un modèle de régression linéaire appliqué à notre dataset ?

In [ ]:
np.sqrt(-linear_cv_scores).mean(), np.sqrt(-linear_cv_scores).std()

Stockez la racine carrée de nos scores négatifs dans une variable `linear_cv_scores`, cette variable nous servira plus tard pour comparer nos modèles :

In [ ]:
linear_cv_scores = np.sqrt(-linear_cv_scores)

## 6.7 Optimisation des hyper-paramètres

Nous allons maintenant chercher le meilleur paramètre de régularisation pour les régressions linéaires régularisées. Commencez par importer la classe [`sklearn.model_selection.GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) :

In [ ]:
from sklearn.model_selection import GridSearchCV

Nous définissons ensuite l'ensemble des coefficients de régularisation que nous allons tester. Appelez la fonction [`numpy.logspace`](https://numpy.org/doc/stable/reference/generated/numpy.logspace.html) avec les paramètres `-2`, `2` et `10` et stockez le résultat dans une variable `regul_coeffs`. Nous avons créé un tableau numpy contenant des nombres uniformément répartis sur une échelle logarithmique (en base 10) entre -2 et 2.

In [ ]:
regul_coeffs = np.logspace(-2, 2, 10)
regul_coeffs

Crééz une liste python nommée `param_grid` contenant un dictionnaire avec une unique clef `alpha` dont la valeur est `regul_coeffs`. Cette liste est une grille d'hyper-paramètres, elle servira à indiquer à `GridSearchCV` que nous souhaitons faire varier le paramètre `alpha` de l'estimateur que nous lui passerons en paramètre (un estimateur scikit-learn peut-être un régresseur ou un classifueur, i.e. : un algo qui permet d'entrainer un modèle, avec la méthode `fit`, et de prédire des valeurs, avec la méthode `predict`). Cette grille pourrait contenir plusieurs hyper-paramètres à faire varier (tous nommés en fonction des paramètres de nos estimateurs), dans ce cas, toutes les combinaisons seraient testées.

In [ ]:
param_grid = [{'alpha': regul_coeffs}]
param_grid

Instanciez la classe `GridSearchCV` (nommez votre instance `grid_search`) avec comme paramètres : une instance de régression Lasso, votre grille de d'hyper-paramètres, la valeur `5` pour le paramètre `cv`, la valeur `neg_mean_squared_error` pour le paramètre `scoring`, la valeur `True` pour le paramètre `return_train_score`.

In [ ]:
grid_search = GridSearchCV(
    lasso, param_grid, cv=5,
    scoring="neg_mean_squared_error"
)

Appelez la méthode `fit` sur votre instance de `GridSearchCV` avec comme paramètres votre dataset d'entrainement et vos cibles d'entrainement. Cet appel vous affiche en sortie les paramètres de votre recherche des meilleurs hyper-paramètres, dont votre grille d'hyper-paramètres.

In [ ]:
grid_search.fit(X_train, y_train)

Affichez les attributs `best_params_`, `best_scores_` et `cv_results_` de votre objet `grid_search`. Attention : nous avons utilisé la métrique de performance `neg_mean_squared_error`.

Quel est le meilleur coefficient de régularisation ? Quel est le score correspondant ? Quel sont la moyenne (simple à trouver) et l'écart type (complexe à trouver, facultatif) de la validation croisée sur les 5 plis de notre grid search pour le meilleur coefficient de régularisation ?

In [ ]:
grid_search.best_params_

In [ ]:
np.sqrt(-grid_search.best_score_)

In [ ]:
grid_search.cv_results_

In [ ]:
min_fold = grid_search.cv_results_['rank_test_score'].argmin()
print(
    np.sqrt(-grid_search.cv_results_['mean_test_score'][min_fold]),
    'std wrong way =', np.sqrt(grid_search.cv_results_['std_test_score'][min_fold]),
    'std proper way =',
    np.std([np.sqrt(-grid_search.cv_results_["split{}_test_score".format(i)][min_fold]) for i in range(5)])
)

`GridSearchCV` fournit directement une instance de notre estimateur avec les meilleurs hyper-paramètres via l'attribut `best_estimator_`. Stockez la sortie de cet attribut dans une variable `best_lasso_model` :

In [ ]:
best_lasso_model = grid_search.best_estimator_

Appliquez une validation croisée sur cet estimateur avec 10 plis et stockez la racine carré des valeurs négatives des scores dans une variable `lasso_cv_scores`, puis affichez la moyenne et l'écart type obtenu :

In [ ]:
scores = cross_val_score(best_lasso_model, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
lasso_cv_scores = np.sqrt(-scores)
np.mean(lasso_cv_scores), np.std(lasso_cv_scores)

Faites de même pour une regression Ridge avec les mêmes valeurs pour le coefficient de régularisation, stockez les scores dans la variable `ridge_cv_scores` :

In [ ]:
param_grid = [{'alpha': regul_coeffs}]
grid_search = GridSearchCV(
    ridge, param_grid, cv=5,
    scoring="neg_mean_squared_error", return_train_score=True
)
grid_search.fit(X_train, y_train)
print('Meilleur coefficient de régularisation : ', grid_search.best_params_["alpha"])
rmse_best_train = np.sqrt(-grid_search.best_score_)
print('rmse entraînement: ', rmse_best_train)
best_ridge_model = grid_search.best_estimator_
scores = cross_val_score(best_ridge_model, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
ridge_cv_scores = np.sqrt(-scores)
np.mean(ridge_cv_scores), np.std(ridge_cv_scores)

## 6.8 Comparaison des modèles

Nous pouvons maintenant comparer les performances de nos modèles objectivement (i.e. : avec des validations croisées sur notre dataset de test) au travers de leurs moyennes et de leurs écart-types. Si vous avez respecté les noms des variables pour les scores des régressions linéaires simple, Ridge et Lasso (sinon, changez le nom des variables sur la première ligne), vous pouvez exécuter la cellule suivante pour avoir un graphique comparant ces trois méthodes.

In [ ]:
model_scores = [linear_cv_scores, lasso_cv_scores, ridge_cv_scores]
model_names = ['Linéaire', 'LASSO', 'ridge']
figure = plt.figure()
axis = figure.add_subplot(111)
plt.boxplot(model_scores)
axis.set_xticklabels(model_names, rotation = 45, ha="right")
axis.set_ylabel("Root Mean Squared Error (RMSE)");

## 6.9 Autres modèles (facultatif)

Testez et comparez les modèles suivants avec les modèles précédents (pensez à recherchez les meilleurs hyper-paramètres indiqués entre parenthèse) :
* [`sklearn.linear_model.ElasticNet`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) (`alpha` et `l1_ratio`)
* [`sklearn.ensemble.RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) (`min_samples_leaf` et `max_features`)

## 6.10 Evaluation du meilleur modèle sur le jeu de test

Vous pouvez maintenant évaluer les performances du meilleur modèle obtenu avec la validation croisée sur le dataset de test :

In [ ]:
from sklearn.metrics import mean_squared_error

test_pred = lr.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
print("Test RMSE : ", test_rmse)

## 6.11 Comparaison de l'effet du coefficient de régularisation pour Ridge et Lasso

Nous construisons plusieurs modèles de régression :
* Une régression linéaire simple.
* Une régression linéaire régularisée de type 'ridge'.
* Une régression linéaire regularisée de type 'LASSO'.

Pour les modèles régularisés, nous testons $10$ valeurs entre $10^{-2}$ et $10^2$ pour le coefficient de régularisation.

Nous traçons l'évolution des poids associés aux variables de la régression linéaire en fonction de la valeur du coefficient de régularisation.

Comment interprétez-vous la différence d'évolution de ces poids pour la régularisation ridge et LASSO ?

Observez-vous des connexions avec l'étude de la corrélation entre les variables ?

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

linear = linear_model.LinearRegression()
linear.fit(X_train, y_train)
pred = linear.predict(X_test)
rmse_linear = np.sqrt(mean_squared_error(y_test, pred))

ridge = linear_model.Ridge()
lasso = linear_model.Lasso()

regul_coefs = np.logspace(-2, 2, 10)

rmse_ridge = []
rmse_lasso = []
params_ridge = []
params_lasso = []

for c in regul_coefs:
    ridge.set_params(alpha=c)
    ridge.fit(X_train, y_train)
    params_ridge.append(ridge.coef_)
    pred = ridge.predict(X_test)
    rmse_ridge.append(np.sqrt(mean_squared_error(y_test, pred)))
    
    lasso.set_params(alpha=c)
    lasso.fit(X_train, y_train)
    params_lasso.append(lasso.coef_)
    pred = lasso.predict(X_test)
    rmse_lasso.append(np.sqrt(mean_squared_error(y_test, pred)))

plt.figure(figsize=(16, 6))
plt.subplot(1,2,1)
lines = plt.semilogx(regul_coefs, params_ridge)
plt.legend(lines, df.columns.values)
plt.xlabel('$C$',fontsize=18)
plt.ylabel('$w_i$',fontsize=18)
plt.title('Ridge')

plt.subplot(1,2,2)
lines = plt.semilogx(regul_coefs, params_lasso)
plt.legend(lines, df.columns.values)
plt.xlabel('$C$',fontsize=18)
plt.ylabel('$w_i$',fontsize=18)
plt.title('Lasso')

plt.show()

### Evaluation de la performance des modèles sur le jeu de test

Que conclure de l'évaluation comparée des modèles sur le jeu de test ? La régularisation est-elle utile ?

In [ ]:
plt.semilogx(regul_coefs, rmse_ridge, 'b', label='RMSE Ridge')
plt.semilogx(regul_coefs, rmse_lasso, 'g', label='RMSE LASSO')
plt.axhline(y=rmse_linear, color='r', linestyle='-', label='RMSE sans régularisation')
plt.legend(loc='best')
plt.xlabel('$C$', fontsize=18)
plt.ylabel('RMSE', fontsize=18)
plt.show()